# Digital Twin Agent

The `DigitalTwinAgent` is a type of perception agent. Its core responsibility is to act as a bridge between a simulated physical object (like a reservoir or a pipe) and the wider multi-agent system. It effectively creates a 'digital twin' of a physical asset within the simulation environment.

## Role and Function

In a real-world system, you would have physical sensors (e.g., a water level sensor) that broadcast their readings. In our simulation, the `DigitalTwinAgent` plays the role of this sensor.

1.  **Wraps a Model:** It holds an instance of a physical object model (e.g., `Reservoir`, `Canal`).
2.  **Publishes State:** At each simulation time step, it queries the state of its wrapped model.
3.  **Broadcasts:** It then publishes this state onto the `MessageBus` on a designated topic.

This allows other agents, such as `LocalControlAgent`s, to subscribe to these state topics and receive the 'sensor' data they need to make decisions, without needing to have direct access to the physical models themselves. This promotes a decoupled and scalable architecture.

## Simulation Example

In this example, we'll create a simple `Reservoir` model. We will then wrap it with a `DigitalTwinAgent`. We will also create a 'monitoring station' that listens for state updates. We will manually advance the reservoir's state and then call the agent's `run()` method, which should trigger a state publication that our monitoring station receives.

**Note:** To run this notebook, you need to be running the Jupyter server from the root directory of the project.

In [ ]:
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.central_coordination.collaboration.message_bus import MessageBus, Message
from swp.core.interfaces import State, Parameters

# 1. A mock monitoring station to listen for state updates
received_states = []
def monitoring_station_callback(message: Message):
    print(f"Monitoring station received state: {message}")
    received_states.append(message)

# 2. Setup the message bus and the physical model
bus = MessageBus()
bus.subscribe('reservoir_A_state_topic', monitoring_station_callback)

reservoir_model = Reservoir(
    name="Reservoir_A",
    initial_state=State(volume=5000, outflow=10),
    parameters=Parameters(surface_area=1000)
)

# 3. Create the Digital Twin Agent
twin_agent = DigitalTwinAgent(
    agent_id="twin_for_reservoir_A",
    simulated_object=reservoir_model,
    message_bus=bus,
    state_topic='reservoir_A_state_topic'
)

# 4. Simulate a time step
print("--- Initial State Publication ---")
# In a real simulation, a harness would call run() periodically
twin_agent.run(current_time=0)

print("\n--- Simulating a change in the reservoir ---")
# Manually update the reservoir's state for demonstration
reservoir_model.set_inflow(20)
reservoir_model.step(action={'outflow': 12}, dt=3600)
print(f"Reservoir's internal state updated to: {reservoir_model.get_state()}")

print("\n--- Second State Publication ---")
# The agent now runs again and publishes the *new* state
twin_agent.run(current_time=3600)

# --- Verification ---
assert len(received_states) == 2
assert received_states[0]['volume'] == 5000
assert received_states[1]['volume'] > 5000 # Check that the updated volume was received
print("\nSuccessfully verified that the agent published the initial and updated states.")